In [11]:
import tensorflow as tf
from spacy.lang.en import English

import tensorflow as tf

import pandas as pd
import spacy
import re
import os
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
curr_path = os.getcwd()
file_path = curr_path + '/dataset/fake_job_postings.csv'
df = pd.read_csv(file_path)
df.fillna(' ',inplace=True)
df['text']=df['title']+" " + df['department'] + " " + df['company_profile'] + " " + df['description'] + " " + df['requirements'] + " " + df['benefits'] + " " 
delete_list=['job_id','title','location','telecommuting','has_company_logo','has_questions','department','salary_range','company_profile','description','requirements','benefits','employment_type','required_experience','required_education','industry','function']
df = df.drop(delete_list, axis=1)

df.head()

,fraudulent,text
0,0,"Marketing Intern Marketing We're Food52, and w..."
1,0,Customer Service - Cloud Video Production Succ...
2,0,Commissioning Machinery Assistant (CMA) Valo...
3,0,Account Executive - Washington DC Sales Our pa...
4,0,Bill Review Manager SpotSource Solutions LLC...


只把第一行拿出来试试，要不太慢

In [8]:
first_row = df.iloc[[0]]

array(["Marketing Intern Marketing We're Food52, and we've created a groundbreaking and award-winning cooking site. We support, connect, and celebrate home cooks, and give them everything they need in one place.We have a top editorial, business, and engineering team. We're focused on using technology to find new and better ways to connect people around their specific food interests, and to offer them superb, highly curated information about food and cooking. We attract the most talented home cooks and contributors in the country; we also publish well-known professionals like Mario Batali, Gwyneth Paltrow, and Danny Meyer. And we have partnerships with Whole Foods Market and Random House.Food52 has been named the best food website by the James Beard Foundation and IACP, and has been featured in the New York Times, NPR, Pando Daily, TechCrunch, and on the Today Show.We're located in Chelsea, in New York City. Food52, a fast-growing, James Beard Award-winning online food community and cro

In [9]:
#Data Cleanups
def data_cleanups(df:pd.DataFrame):
    df['text']=df['text'].str.replace('\n','')
    df['text']=df['text'].str.replace('\r','')
    df['text']=df['text'].str.replace('\t','')
  
  #This removes unwanted texts
    df['text'] = df['text'].apply(lambda x: re.sub(r'[0-9]','',x))
    df['text'] = df['text'].apply(lambda x: re.sub(r'[/(){}\[\]\|@,;.:-]',' ',x))
  
  #Converting all upper case to lower case
    df['text']= df['text'].apply(lambda s:s.lower() if type(s) == str else s)
  
  #Remove un necessary white space
    df['text']=df['text'].str.replace('  ',' ')

  #Remove Stop words
    nlp=spacy.load("en_core_web_sm") # python -m spacy download en_core_web_sm 
    df['text'] =df['text'].apply(lambda x: ' '.join([word for word in x.split() if nlp.vocab[word].is_stop==False ]))
    
    return df

first_row = data_cleanups(first_row)
first_row

C:\Users\11311\AppData\Local\Temp\ipykernel_5984\4242022306.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text']=df['text'].str.replace('\n','')
C:\Users\11311\AppData\Local\Temp\ipykernel_5984\4242022306.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text']=df['text'].str.replace('\r','')
C:\Users\11311\AppData\Local\Temp\ipykernel_5984\4242022306.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

,fraudulent,text
0,0,marketing intern marketing we're food we've cr...


In [10]:
sp = spacy.load('en_core_web_sm')
output=[]

for sentence in first_row['text']:
    sentence=sp(str(sentence))
    s=[token.lemma_ for token in sentence]
    output.append(' '.join(s))
first_row['processed']=pd.Series(output)

first_row

C:\Users\11311\AppData\Local\Temp\ipykernel_5984\1179493015.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_row['processed']=pd.Series(output)


,fraudulent,text,processed
0,0,marketing intern marketing we're food we've cr...,market intern marketing we be food we have cre...


用新的tokenizer是返回的NP array

In [ ]:
vocab_size = 100000
embedding_dim = 64
max_length = 250
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000
#Tokenization

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(first_row['processed'].values)
word_index = tokenizer.word_index

X = tokenizer.texts_to_sequences(first_row['processed'].values)    # Tokenize the dataset
X = pad_sequences(X, maxlen=max_length)   # Padding the dataset
Y = first_row['fraudulent']       #Assign the value of y  
X

In [19]:
X

array([[  0,   0,  13,  14,  15,   2,   6,   1,   2,  35,  36,  37,  16,
         17,   7,  38,   8,  18,  39,   9,   7,  40,  41,  19,  42,  43,
         20,   2,   6,  44,  45,  46,   3,  47,  48,  18,  49,  50,   1,
         51,  52,  53,  54,  21,  55,   1,  10,  56,  57,   9,   7,  58,
         59,  60,  61,  62,  22,  63,  64,  65,  66,  67,  68,  69,   1,
         13,  70,  71,   1,  72,  73,   1,  23,  24,  25,  74,  75,  76,
          3,  11,  77,  78,  79,  80,  81,  82,   2,   6,  83,  84,   3,
         11,  26,   1,  85,  86,  24,  25,  16,  17,  87,   1,  88,  89,
         90,  21,  91,  92,  93,  94,  95,  96,  14,   4,  27,  20,  28,
         29,  30,   3,  11,  26,  97,  98,  99, 100,   1,  12, 101, 102,
        103, 104, 105, 106, 107,  12,  31, 108, 109,  23, 110,   1,   5,
        111,  32,  32,   5, 112,   8, 113,   5, 114,   5, 115, 116, 117,
        118, 119, 120,  33,   4, 121, 122, 123, 124,  30, 125, 126, 127,
        128, 129,  15,  29, 130, 131,  12,  31, 132

来试试用老的tokenizer

In [17]:
import pickle
def read_tokenizer_wordIndex():
    with open('tokenizer.pickle', 'rb') as handle:
        loaded_tokenizer = pickle.load(handle)
    # 加载 word_index
    with open('word_index.pickle', 'rb') as handle:
        loaded_word_index = pickle.load(handle)
        
    return loaded_tokenizer, loaded_word_index

In [18]:
loaded_tokenizer, loaded_word_index = read_tokenizer_wordIndex()

X_new = tokenizer.texts_to_sequences(first_row['processed'].values)    # Tokenize the dataset
X_new = pad_sequences(X_new, maxlen=max_length)   # Padding the dataset
X_new

array([[  0,   0,  13,  14,  15,   2,   6,   1,   2,  35,  36,  37,  16,
         17,   7,  38,   8,  18,  39,   9,   7,  40,  41,  19,  42,  43,
         20,   2,   6,  44,  45,  46,   3,  47,  48,  18,  49,  50,   1,
         51,  52,  53,  54,  21,  55,   1,  10,  56,  57,   9,   7,  58,
         59,  60,  61,  62,  22,  63,  64,  65,  66,  67,  68,  69,   1,
         13,  70,  71,   1,  72,  73,   1,  23,  24,  25,  74,  75,  76,
          3,  11,  77,  78,  79,  80,  81,  82,   2,   6,  83,  84,   3,
         11,  26,   1,  85,  86,  24,  25,  16,  17,  87,   1,  88,  89,
         90,  21,  91,  92,  93,  94,  95,  96,  14,   4,  27,  20,  28,
         29,  30,   3,  11,  26,  97,  98,  99, 100,   1,  12, 101, 102,
        103, 104, 105, 106, 107,  12,  31, 108, 109,  23, 110,   1,   5,
        111,  32,  32,   5, 112,   8, 113,   5, 114,   5, 115, 116, 117,
        118, 119, 120,  33,   4, 121, 122, 123, 124,  30, 125, 126, 127,
        128, 129,  15,  29, 130, 131,  12,  31, 132

In [20]:
X is X_new

False